# iPhone X Reviews: Sentiment Analysis

## Opinion Mining

In [1]:
from lxml import html
import requests
import re

url='https://www.facebook.com/plugins/feedback.php?api_key=1678638095724206&channel_url=https%3A%2F%2Fstaticxx.facebook.com%2Fconnect%2Fxd_arbiter%2Fr%2FhsBwMj6iLmk.js%3Fversion%3D42%23cb%3Df5484fdd8d9488%26domain%3Dtechcrunch.com%26origin%3Dhttps%253A%252F%252Ftechcrunch.com%252Ff7f8195ccf0fa%26relation%3Dparent.parent&href=https%3A%2F%2Ftechcrunch.com%2F2017%2F09%2F13%2Fi-dont-want-the-new-iphone-x-and-i-cant-be-alone%2F&locale=en_US&numposts=100&sdk=joey&version=v2.6&width=100%25&refsrc=https%3A%2F%2Fm.facebook.com%2Fplugins%2Fcomments.php&_rdr'
page = requests.get(url)
tree = html.fromstring(page.content)
comments_section = tree.xpath('///text()')


part1=re.split('text":"',comments_section[20])
comments_list=[]

for i in range(3,len(part1),2):
    part2=re.split('"},"ranges"',part1[i])
    comment=str(part2[0])
    comment= comment.replace("\\u2019","'")
    comment= comment.replace("\\u00a3","£")
    comment= comment.replace("u0025","%")
    comment= comment.replace("u0040","@")
    comment= comment.replace("\nn"," ")
    comment= comment.replace("\n"," ")
    comment= comment.replace("\t"," ")
    comment= comment.replace('\\', '')
    comment= comment.replace('.t', '.')
    comment= comment.replace('nn', '')
    comment= comment.replace('n-n-', '')
    comment= comment.replace('-n-n', '')
    comment= comment.replace('. n', ' ')
    comment= comment.replace('.  n', ' ')
    comment= comment.replace('.n', ' ')
    comment= comment.replace('?n', ' ')
    comment= comment.replace('\'',"’")
    
    comments_list.append(comment)

In [2]:
# if len(comments_list)==0, use comments_section[21] in the definition of part1
len(comments_list)

128

In [3]:
# Three sample comments
comments_list[:3]

['The most useful and accurate thing you said was "I understand how this editorial makes me look. I’m that guy. I’m the guy at the dealership yelling at a green salesmen because the new Buick does not have an ashtray." Yeah. You’re that guy. And you continue to be that guy. The guy who walks through life never learning new lessons. Good luck with that.',
 'Why spend $1000 every 2 years on a phone, when you can get last years model ... just as good in all things that matter, at almost half that price.  Those new features are lame.  Wireless buds = powerful battery inside my head.  No thanks.  Bezel-less screen ... I care because of why?  The bezel has never been a problem by the way, you don’t even see it with a phone case!  No wonder Apple is so rich.  They’ve somehow tapped into the most bilkable people in the world.',
 'You can’t be alone and you’re not alone.  I keep reading reviews of the iphone 8 that say "who is this phone for?"  Um... me?  I’m giving my old iphone to my daughter

## Data Prep for Sentiment Analysis

In [4]:
import pandas as pd
import numpy as np

# Read in the data
df1 = pd.read_csv('https://raw.githubusercontent.com/ehsansade/Sentiment-Analysis/master/Amazon_Unlocked_Mobile_Reviews1.csv')
df2 = pd.read_csv('https://raw.githubusercontent.com/ehsansade/Sentiment-Analysis/master/Amazon_Unlocked_Mobile_Reviews2.csv')

df=pd.concat([df1,df2],ignore_index=True)


df.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,Apple iPhone SE Unlocked Phone - 64 GB Retail ...,Apple,514.98,5,"Exactly what I wanted, fast shipping.",0.0
1,Apple iPhone SE Unlocked Phone - 64 GB Retail ...,Apple,514.98,5,I was a little skeptical about buying electron...,1.0
2,Apple iPhone SE Unlocked Phone - 64 GB Retail ...,Apple,514.98,1,"Phone was not what I ordered, very disappointe...",1.0
3,Apple iPhone SE Unlocked Phone - 64 GB Retail ...,Apple,514.98,5,The sexy body of a 5s and the guts of a 6........,0.0
4,Apple iPhone SE Unlocked Phone - 64 GB Retail ...,Apple,514.98,5,Love it!,0.0


In [5]:
# Drop missing values
df.dropna(inplace=True)

# Remove any 'neutral' ratings equal to 3
df = df[df['Rating'] != 3]

# Encode 4s and 5s as 1 (rated positively)
# Encode 1s and 2s as 0 (rated poorly)
df['Positively Rated'] = np.where(df['Rating'] > 3, 1, 0)
df.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes,Positively Rated
0,Apple iPhone SE Unlocked Phone - 64 GB Retail ...,Apple,514.98,5,"Exactly what I wanted, fast shipping.",0.0,1
1,Apple iPhone SE Unlocked Phone - 64 GB Retail ...,Apple,514.98,5,I was a little skeptical about buying electron...,1.0,1
2,Apple iPhone SE Unlocked Phone - 64 GB Retail ...,Apple,514.98,1,"Phone was not what I ordered, very disappointe...",1.0,0
3,Apple iPhone SE Unlocked Phone - 64 GB Retail ...,Apple,514.98,5,The sexy body of a 5s and the guts of a 6........,0.0,1
4,Apple iPhone SE Unlocked Phone - 64 GB Retail ...,Apple,514.98,5,Love it!,0.0,1


In [6]:
from sklearn.model_selection import train_test_split

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df['Reviews'], 
                                                    df['Positively Rated'],random_state=3)

## CountVectorizer, n-grams

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(min_df=5, ngram_range=(1,2)).fit(X_train)

X_train_vectorized = vect.transform(X_train)

len(vect.get_feature_names())

103120

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

predictions = model.predict(vect.transform(X_test))

print('ROC_AUC: ', roc_auc_score(y_test, predictions))

ROC_AUC:  0.969177058414


In [9]:
feature_names = np.array(vect.get_feature_names())

sorted_coef_index = model.coef_[0].argsort()

print('Smallest Coefs:\n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}'.format(feature_names[sorted_coef_index[:-11:-1]]))

Smallest Coefs:
['no good' 'junk' 'worst' 'poor' 'looks ok' 'not good' 'horrible'
 'terrible' 'not happy' 'not very']

Largest Coefs: 
['excellent' 'excelent' 'excelente' 'not bad' 'no problems' 'perfect'
 'awesome' 'exelente' 'great' 'amazing']


In [10]:
# check the model to see if it correctly identifies a positive review
print(model.predict(vect.transform(['not an issue, phone is working',
                                    'an issue, phone is not working'])))

[1 0]


## Sentiment Analysis for iPhone X

In [11]:
comments_positivity=model.predict(vect.transform(comments_list))
print(comments_positivity)

[1 1 1 1 0 1 1 0 1 1 0 1 1 1 0 0 0 0 0 0 0 1 1 1 0 1 0 0 0 0 1 1 0 0 0 1 1
 1 0 0 1 1 0 0 1 1 0 0 1 1 1 1 1 1 0 0 1 0 1 1 1 1 0 0 1 1 0 1 1 1 1 1 1 0
 0 1 0 0 0 0 0 0 1 0 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 0 1 0 1 1 1 1 1
 0 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0]


In [12]:
comments_positivity.mean()

0.609375